# Smart Vending Insights: Data Analysis Project

## Overview
This notebook analyzes vending machine sales data to uncover insights about product performance, machine efficiency, and customer behavior patterns. The goal is to understand what drives sales and identify optimization opportunities.

**Dataset**: 3 months of vending machine sales data (500+ transactions)
**Tools**: Pandas, NumPy, Matplotlib, Seaborn
**Focus**: Revenue analysis, demand patterns, inventory optimization

## 1. Data Loading and Initial Exploration

Let's start by loading our dataset and understanding its structure.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
df = pd.read_csv('../data/vending_sales.csv')

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"\nColumn types:")
print(df.dtypes)
print(f"\nFirst few rows:")
df.head()

## 2. Data Cleaning and Feature Engineering

Now we'll clean the data and create useful derived features for our analysis.

In [ ]:
# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Create derived features
df['date'] = df['timestamp'].dt.date
df['day_of_week'] = df['timestamp'].dt.day_name()
df['hour'] = df['timestamp'].dt.hour
df['revenue'] = df['price'] * df['quantity']

# Check for any data quality issues
print(f"Missing values:")
print(df.isnull().sum())

print(f"\nData validation - Stock consistency:")
stock_check = (df['stock_after'] == df['stock_before'] - df['quantity']).all()
print(f"Stock calculations consistent: {stock_check}")

print(f"\nDataset summary after cleaning:")
df.info()

## 3. Revenue Analysis

Let's explore the revenue patterns and understand which products and machines are performing best.

In [ ]:
# Basic revenue metrics
total_revenue = df['revenue'].sum()
avg_order_value = df['revenue'].mean()
total_transactions = len(df)

print(f"=== REVENUE SUMMARY ===")
print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Average Order Value: ${avg_order_value:.2f}")
print(f"Total Transactions: {total_transactions:,}")

# Revenue by product
revenue_by_product = df.groupby('product')['revenue'].sum().sort_values(ascending=False)
print(f"\n=== REVENUE BY PRODUCT ===")
for product, revenue in revenue_by_product.items():
    percentage = (revenue / total_revenue) * 100
    print(f"{product}: ${revenue:.2f} ({percentage:.1f}%)")

# Revenue by machine
revenue_by_machine = df.groupby('machine_id')['revenue'].sum().sort_values(ascending=False)
print(f"\n=== REVENUE BY MACHINE ===")
for machine, revenue in revenue_by_machine.items():
    percentage = (revenue / total_revenue) * 100
    print(f"{machine}: ${revenue:.2f} ({percentage:.1f}%)")

# Revenue by location type
revenue_by_location = df.groupby('location_type')['revenue'].sum().sort_values(ascending=False)
print(f"\n=== REVENUE BY LOCATION ===")
for location, revenue in revenue_by_location.items():
    percentage = (revenue / total_revenue) * 100
    print(f"{location}: ${revenue:.2f} ({percentage:.1f}%)")

## 4. Demand Pattern Analysis

Understanding when and how demand varies helps with inventory planning and staffing decisions.

In [ ]:
# Daily revenue analysis
daily_revenue = df.groupby('date')['revenue'].sum().reset_index()
daily_revenue['date'] = pd.to_datetime(daily_revenue['date'])

# 7-day rolling averages per machine
machine_daily = df.groupby(['machine_id', 'date'])['quantity'].sum().reset_index()
machine_daily['date'] = pd.to_datetime(machine_daily['date'])

rolling_averages = {}
for machine in df['machine_id'].unique():
    machine_data = machine_daily[machine_daily['machine_id'] == machine].set_index('date')
    machine_data = machine_data.reindex(pd.date_range(machine_data.index.min(), machine_data.index.max(), freq='D'), fill_value=0)
    rolling_avg = machine_data['quantity'].rolling(window=7, min_periods=1).mean()
    rolling_averages[machine] = rolling_avg

print(f"=== DEMAND PATTERNS ===")

# Hour of day analysis
hourly_sales = df.groupby('hour')['quantity'].sum()
peak_hour = hourly_sales.idxmax()
print(f"Peak sales hour: {peak_hour}:00 ({hourly_sales[peak_hour]} items sold)")

# Day of week analysis
daily_sales = df.groupby('day_of_week')['quantity'].sum()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_sales = daily_sales.reindex(day_order)
peak_day = daily_sales.idxmax()
print(f"Peak sales day: {peak_day} ({daily_sales[peak_day]} items sold)")

# Weather impact
weather_sales = df.groupby('rain').agg({'quantity': 'sum', 'temperature_c': 'mean'})
print(f"\n=== WEATHER IMPACT ===")
print(f"Rainy days - Total sales: {weather_sales.loc[1, 'quantity']}, Avg temp: {weather_sales.loc[1, 'temperature_c']:.1f}°C")
print(f"Dry days - Total sales: {weather_sales.loc[0, 'quantity']}, Avg temp: {weather_sales.loc[0, 'temperature_c']:.1f}°C")

## 5. Inventory Optimization Analysis

Let's analyze stock levels and calculate optimal reorder points and safety stock levels.

In [ ]:
# Calculate demand statistics for each machine-product combination
demand_stats = df.groupby(['machine_id', 'product'])['quantity'].agg(['mean', 'std', 'count']).reset_index()
demand_stats.columns = ['machine_id', 'product', 'avg_demand', 'demand_std', 'frequency']

# Calculate reorder point and safety stock
# Assuming 3-day lead time and 95% service level (Z = 1.65)
LEAD_TIME_DAYS = 3
SERVICE_LEVEL_Z = 1.65

demand_stats['daily_demand'] = demand_stats['avg_demand'] * demand_stats['frequency'] / 90  # 90 days of data
demand_stats['lead_time_demand'] = demand_stats['daily_demand'] * LEAD_TIME_DAYS
demand_stats['safety_stock'] = SERVICE_LEVEL_Z * demand_stats['demand_std'] * np.sqrt(LEAD_TIME_DAYS)
demand_stats['reorder_point'] = demand_stats['lead_time_demand'] + demand_stats['safety_stock']

# Round values for practical implementation
demand_stats['safety_stock'] = np.ceil(demand_stats['safety_stock'].fillna(0))
demand_stats['reorder_point'] = np.ceil(demand_stats['reorder_point'].fillna(0))

print(f"=== INVENTORY OPTIMIZATION RECOMMENDATIONS ===")
print(f"
Top 10 Machine-Product combinations by daily demand:")
top_demand = demand_stats.nlargest(10, 'daily_demand')[['machine_id', 'product', 'daily_demand', 'reorder_point', 'safety_stock']]
print(top_demand.to_string(index=False))

# Detect potential stockouts
current_stock = df.groupby(['machine_id', 'product'])['stock_after'].last().reset_index()
stock_analysis = current_stock.merge(demand_stats[['machine_id', 'product', 'reorder_point']], on=['machine_id', 'product'])
stockout_risk = stock_analysis[stock_analysis['stock_after'] <= stock_analysis['reorder_point']]

print(f"\n=== STOCKOUT ALERTS ===")
if len(stockout_risk) > 0:
    print(f"⚠️  {len(stockout_risk)} machine-product combinations need restocking:")
    for _, row in stockout_risk.iterrows():
        print(f"{row['machine_id']} - {row['product']}: Current stock {row['stock_after']}, Reorder at {row['reorder_point']}")
else:
    print(f"✅ All machines have adequate stock levels")

## 6. Price Sensitivity Analysis

Let's simulate the impact of a price change on revenue to understand price elasticity.

In [ ]:
# Simulate 10% price increase for Soda
PRODUCT_TO_TEST = 'Soda'
PRICE_INCREASE = 0.10

# Current performance
current_soda_sales = df[df['product'] == PRODUCT_TO_TEST]
current_units = current_soda_sales['quantity'].sum()
current_revenue = current_soda_sales['revenue'].sum()
current_price = current_soda_sales['price'].iloc[0]

# Simulate demand elasticity (assume -0.5 elasticity: 10% price increase = 5% demand decrease)
DEMAND_ELASTICITY = -0.5
demand_change = DEMAND_ELASTICITY * PRICE_INCREASE
new_demand_multiplier = 1 + demand_change

# Calculate new scenario
new_price = current_price * (1 + PRICE_INCREASE)
new_units = current_units * new_demand_multiplier
new_revenue = new_units * new_price

revenue_impact = new_revenue - current_revenue
revenue_change_pct = (revenue_impact / current_revenue) * 100

print(f"=== PRICE SENSITIVITY ANALYSIS: {PRODUCT_TO_TEST} ===")
print(f"
Current Scenario:")
print(f"  Price: ${current_price:.2f}")
print(f"  Units sold: {current_units}")
print(f"  Revenue: ${current_revenue:.2f}")

print(f"
With {PRICE_INCREASE*100}% price increase:")
print(f"  New price: ${new_price:.2f}")
print(f"  Estimated units: {new_units:.0f} ({demand_change*100:+.1f}%)")
print(f"  Estimated revenue: ${new_revenue:.2f}")
print(f"  Revenue impact: ${revenue_impact:+.2f} ({revenue_change_pct:+.1f}%)")

if revenue_impact > 0:
    print(f"💡 Recommendation: Price increase would be profitable!")
else:
    print(f"⚠️  Recommendation: Price increase would reduce overall revenue.")

## 7. Data Visualizations

Let's create visualizations to better understand our findings.

In [ ]:
# Set up the plotting style
plt.style.use('default')
sns.set_palette("husl")

# Create a figure with multiple subplots
fig = plt.figure(figsize=(20, 15))

# 1. Top Products by Revenue (Bar Chart)
plt.subplot(2, 3, 1)
revenue_by_product.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Revenue by Product', fontsize=14, fontweight='bold')
plt.xlabel('Product')
plt.ylabel('Revenue ($)')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, v in enumerate(revenue_by_product):
    plt.text(i, v + 1, f'${v:.0f}', ha='center', fontweight='bold')

# 2. Daily Revenue per Machine (Line Chart)
plt.subplot(2, 3, 2)
daily_machine_revenue = df.groupby(['date', 'machine_id'])['revenue'].sum().reset_index()
daily_machine_revenue['date'] = pd.to_datetime(daily_machine_revenue['date'])

for machine in df['machine_id'].unique():
    machine_data = daily_machine_revenue[daily_machine_revenue['machine_id'] == machine]
    plt.plot(machine_data['date'], machine_data['revenue'], marker='o', linewidth=2, label=machine)

plt.title('Daily Revenue by Machine', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Daily Revenue ($)')
plt.legend()
plt.xticks(rotation=45)
plt.grid(alpha=0.3)

# 3. Sales Heatmap: Hour vs Product
plt.subplot(2, 3, 3)
heatmap_data = df.pivot_table(values='quantity', index='hour', columns='product', aggfunc='sum', fill_value=0)
sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='YlOrRd', cbar_kws={'label': 'Units Sold'})
plt.title('Sales Heatmap: Hour vs Product', fontsize=14, fontweight='bold')
plt.xlabel('Product')
plt.ylabel('Hour of Day')

# 4. Revenue by Location Type
plt.subplot(2, 3, 4)
revenue_by_location.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title('Revenue Distribution by Location', fontsize=14, fontweight='bold')
plt.ylabel('')

# 5. Hourly Sales Pattern
plt.subplot(2, 3, 5)
hourly_sales.plot(kind='bar', color='lightgreen', edgecolor='black')
plt.title('Sales by Hour of Day', fontsize=14, fontweight='bold')
plt.xlabel('Hour')
plt.ylabel('Units Sold')
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)

# 6. Temperature vs Sales
plt.subplot(2, 3, 6)
plt.scatter(df['temperature_c'], df['quantity'], alpha=0.6, color='coral')

# Add trend line
z = np.polyfit(df['temperature_c'], df['quantity'], 1)
p = np.poly1d(z)
plt.plot(df['temperature_c'].sort_values(), p(df['temperature_c'].sort_values()), "r--", alpha=0.8, linewidth=2)

plt.title('Temperature vs Sales Volume', fontsize=14, fontweight='bold')
plt.xlabel('Temperature (°C)')
plt.ylabel('Quantity Sold')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('vending_analysis_charts.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Visualizations saved as 'vending_analysis_charts.png'")

## 8. Key Insights and Recommendations

Based on our analysis, here are the main findings and actionable insights.

In [ ]:
# Summary insights
print("=== 🔍 KEY INSIGHTS ===\n")

print("1. 📈 REVENUE PERFORMANCE")
top_product = revenue_by_product.index[0]
top_product_pct = (revenue_by_product.iloc[0] / total_revenue) * 100
print(f"   • {top_product} is the top revenue generator ({top_product_pct:.1f}% of total sales)")

top_machine = revenue_by_machine.index[0]
top_machine_pct = (revenue_by_machine.iloc[0] / total_revenue) * 100
print(f"   • Machine {top_machine} generates the most revenue ({top_machine_pct:.1f}% of total)")

print(f"\n2. ⏰ DEMAND PATTERNS")
print(f"   • Peak sales occur at {peak_hour}:00 ({hourly_sales[peak_hour]} units)")
print(f"   • {peak_day} is the busiest day ({daily_sales[peak_day]} units sold)")

# Temperature correlation
temp_corr = df['temperature_c'].corr(df['quantity'])
print(f"   • Temperature correlation with sales: {temp_corr:.3f}")

print(f"\n3. 📦 INVENTORY INSIGHTS")
stockout_count = len(stockout_risk)
if stockout_count > 0:
    print(f"   • {stockout_count} machine-product combinations need immediate restocking")
else:
    print(f"   • All machines currently have adequate stock levels")

avg_safety_stock = demand_stats['safety_stock'].mean()
print(f"   • Average recommended safety stock: {avg_safety_stock:.1f} units")

print(f"\n4. 💰 PRICING OPPORTUNITIES")
if revenue_impact > 0:
    print(f"   • {PRODUCT_TO_TEST} price increase could generate additional ${revenue_impact:.2f} revenue")
else:
    print(f"   • {PRODUCT_TO_TEST} appears price-sensitive; focus on volume over margin")

print(f"\n=== 🎯 ACTIONABLE RECOMMENDATIONS ===\n")

print(f"1. 🔄 OPERATIONAL OPTIMIZATION")
print(f"   • Schedule restocking during off-peak hours (avoid {peak_hour}:00)")
print(f"   • Increase {top_product} inventory across all machines")
print(f"   • Focus promotional efforts on {peak_day}s")

print(f"\n2. 📊 INVENTORY MANAGEMENT")
print(f"   • Implement automated reorder alerts based on calculated reorder points")
print(f"   • Maintain safety stock levels to prevent stockouts")
print(f"   • Monitor {top_machine} more closely due to high sales volume")

print(f"\n3. 💡 GROWTH OPPORTUNITIES")
if temp_corr > 0.1:
    print(f"   • Consider weather-based inventory adjustments (higher temp = more sales)")
elif temp_corr < -0.1:
    print(f"   • Stock more hot beverages during cooler weather")
else:
    print(f"   • Weather has minimal impact on sales patterns")

print(f"   • Test premium pricing for high-demand products during peak hours")
print(f"   • Consider expanding successful machine locations to similar venues")